In [ ]:
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [6]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
    parse_only=bs4.SoupStrainer(
        class_=("post-content", "post-title", "post-header")
    )
    ),
)
doc = loader.load()

#Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50,
)

splits = text_splitter.split_documents(doc)

#Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()


### 1. Multi Query

The multi-query technique involves generating multiple versions of a user question to retrieve relevant documents from a vector database. This approach helps overcome some limitations of distance-based similarity search by providing multiple perspectives on the user question. 

In [13]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. 
Original question: {question}"""
multi_query = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    multi_query 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [14]:
question = "What is task decomposition for LLM agents?"
queries = generate_queries.invoke({"question": question})
queries

['1. How do LLM agents utilize task decomposition in their operations?',
 '2. Can you explain the concept of task decomposition as applied to LLM agents?',
 '3. In what ways do LLM agents benefit from task decomposition?',
 '4. What role does task decomposition play in the functioning of LLM agents?',
 '5. How is task decomposition integrated into the workflow of LLM agents?']

In [9]:
# Retrieve documents for each query
retrieved_docs = [retriever.invoke( q ) for q in queries]
retrieved_docs

[[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first searc

In [10]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

docs = get_unique_union(retrieved_docs)
docs

C:\Users\Sorou\AppData\Local\Temp\ipykernel_16144\2743590365.py:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)\nThe system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.\nInstruction:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what t

In [ ]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
import pprint

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

multi_query_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_response =  multi_query_chain.invoke({"question":question, "context": docs})
pprint.pprint(final_response)

('Task decomposition for LLM agents involves breaking down complex tasks into '
 'smaller and simpler steps to enhance model performance. This can be achieved '
 'through techniques like Chain of Thought (CoT) and Tree of Thoughts, which '
 'prompt the model to think step by step and explore multiple reasoning '
 'possibilities at each step. Task decomposition can be done using simple '
 'prompting, task-specific instructions, or with human inputs.')


### 2. RAG Fusion

1. Implements RAG Fusion to combine retrieval results from multiple query formulations.
2. Uses Reciprocal Rank Fusion (RRF) to balance document rankings across different queries.
3. Ensures robust document reranking for improved accuracy in RAG pipelines.

In [12]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [16]:
question = "What is task decomposition for LLM agents?"
queries = generate_queries.invoke({"question": question})
queries

['1. How do LLM agents use task decomposition in problem-solving?',
 '2. Benefits of task decomposition for LLM agents',
 '3. Examples of task decomposition strategies for LLM agents',
 '4. Challenges of implementing task decomposition for LLM agents']

In [19]:
retrieved_docs = [retriever.invoke(q) for q in queries]
retrieved_docs

[[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first searc

In [17]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [21]:
reranked_results = reciprocal_rank_fusion(retrieved_docs)
reranked_results

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first searc

In [22]:
# Extract final documents (removing scores)
docs = [doc for doc, _ in reranked_results]
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [23]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

fusion_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_response =  fusion_rag_chain.invoke({"question":question, "context": docs})
pprint.pprint(final_response)

('Task decomposition for LLM agents involves breaking down complex tasks into '
 'smaller and simpler steps using techniques such as Chain of Thought (CoT) '
 'and Tree of Thoughts. This allows the agent to utilize more test-time '
 'computation and enhance its performance on complex tasks by transforming big '
 'tasks into multiple manageable tasks. Task decomposition can be achieved '
 'through simple prompting, task-specific instructions, or human inputs.')


### 3. Query Decomposition

In [24]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [56]:
# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = (
    prompt_decomposition 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)
# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']

#### 3.1 Answer Recursivly

In [33]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [40]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    return f"Question: {question}\nAnswer: {answer}\n"

def run_rag_chain(q, q_a_pairs):
    """Retrieve relevant documents and generate an answer using LLM"""

    # Retrieve relevant context
    retrieved_context = retriever.invoke(q)

    # Format prompt with retrieved context, question, and past Q&A pairs
    formatted_prompt = decomposition_prompt.format(
        context=retrieved_context,
        question=q,
        q_a_pairs=q_a_pairs
    )
    # Generate response using LLM
    response = llm.invoke(formatted_prompt)

    # Parse response to clean output
    parsed_answer = StrOutputParser().invoke(response)

    return parsed_answer

# Initialize LLM model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize empty Q&A history
q_a_pairs = ""

# Process each question in sequence
for q in questions:
    answer = run_rag_chain(q, q_a_pairs)  # Generate answer
    q_a_pair = format_qa_pair(q, answer)  # Format Q&A pair
    q_a_pairs += "\n---\n" + q_a_pair  # Append to history

In [41]:
pprint.pprint(answer)

('In an LLM-powered autonomous agent system, the main components interact with '
 'each other in a coordinated manner to enable autonomous behavior. \n'
 '\n'
 '1. **Planning**: The agent breaks down large tasks into smaller, manageable '
 'subgoals through task decomposition. This step allows the agent to plan '
 'ahead and know the steps required to achieve a specific goal.\n'
 '\n'
 '2. **Subgoal Decomposition**: The agent further decomposes tasks into '
 'smaller steps, making problem-solving more manageable. Techniques like Chain '
 'of Thought (CoT) and Tree of Thoughts help in breaking down complex tasks '
 'into simpler components.\n'
 '\n'
 '3. **Reflection and Refinement**: The agent engages in self-criticism and '
 'self-reflection over past actions, learning from mistakes, and refining its '
 'actions for future steps. This iterative process helps improve the quality '
 "of the agent's decision-making and behavior over time.\n"
 '\n'
 '4. **Memory**: The system may incorpor

#### 3.2 Answer Individually



In [57]:

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(prompt_rag,questions):
    """RAG on each sub-question"""

    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for question in questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(question)
        
        # Step 1: Format the RAG prompt with retrieved context and question
        formatted_prompt = prompt_rag.format(
            context=retrieved_docs,
            question=question
        )

        # Step 2: Pass formatted prompt to the LLM
        response = llm.invoke(formatted_prompt)

        # Step 3: Parse the response into a clean string
        answer = StrOutputParser().invoke(response)
        rag_results.append(answer)
    
    return rag_results,questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(prompt_rag, questions)

C:\Users\Sorou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [61]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

pprint.pprint(final_rag_chain.invoke({"context":context,"question":question}))

('The main components of an LLM-powered autonomous agent system include large '
 'language models (LLM) as the core controller, planning for task breakdown, '
 'subgoal decomposition for efficient handling, reflection for learning from '
 'past actions, and memory for storing information and experiences. These '
 'components work together to enable autonomous behavior by facilitating task '
 'decomposition, self-criticism, and efficient decision-making based on past '
 'experiences.')


### 4. Step Back

In [62]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [65]:
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What is task decomposition in general?'

In [67]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

# Function to generate the final response using the LLM
def generate_response(question):
    """Retrieve context, format prompt, and generate response."""

    # Retrieve normal and step-back contexts
    normal_context = retriever.invoke(question)
    step_back_queries = generate_queries_step_back.invoke( question)
    step_back_context = retriever.invoke( step_back_queries)

    # Format the response prompt
    formatted_prompt = response_prompt_template.format(
        normal_context=normal_context,
        step_back_context=step_back_context,
        question=question
    )

    # Get LLM response
    response = ChatOpenAI(temperature=0).invoke(formatted_prompt)

    # Parse the response
    return StrOutputParser().invoke(response)

# Call the function
final_answer = generate_response(question)

In [68]:
pprint.pprint(final_answer)

('Task decomposition for LLM agents refers to the process of breaking down '
 'complex tasks into smaller and more manageable subtasks or steps. This '
 'approach allows the agent to better understand the overall task and plan its '
 'actions accordingly. In the context of LLM-powered autonomous agents, task '
 'decomposition is crucial for enhancing model performance on complex tasks.\n'
 '\n'
 'One common technique used for task decomposition is the Chain of Thought '
 '(CoT), where the model is instructed to "think step by step" to decompose '
 'hard tasks into simpler steps. This technique transforms big tasks into '
 "multiple manageable tasks and provides insights into the model's thinking "
 'process.\n'
 '\n'
 'Another approach, known as Tree of Thoughts, extends CoT by exploring '
 'multiple reasoning possibilities at each step. It decomposes the problem '
 'into multiple thought steps and generates multiple thoughts per step, '
 'creating a tree structure. The search process 

### 4. HyDE

In [71]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)
# # Format the prompt with the question
# formatted_prompt = prompt_hyde.format(question=question)

# # Generate a response using LLM
# response = ChatOpenAI(temperature=0).invoke(formatted_prompt)

# # Parse the response into clean output
# StrOutputParser().invoke(response)
# Run
question = "What is task decomposition for LLM agents?"
generated_doc = generate_docs_for_retrieval.invoke({"question":question})
pprint.pprint(generated_doc)

('Task decomposition is a fundamental concept in the field of reinforcement '
 'learning and artificial intelligence, particularly for Large Language Models '
 '(LLMs) agents. Task decomposition refers to the process of breaking down a '
 'complex task into smaller, more manageable sub-tasks or components. This '
 'allows the agent to effectively tackle the overall task by focusing on '
 'individual components sequentially or in parallel.\n'
 '\n'
 'In the context of LLM agents, task decomposition is crucial for improving '
 "the efficiency and effectiveness of the agent's decision-making process. By "
 'breaking down a complex task into smaller sub-tasks, the agent can better '
 'understand the underlying structure of the task and identify the most '
 'relevant information needed to make informed decisions. This can lead to '
 'more accurate and timely responses to user queries or commands.\n'
 '\n'
 'Furthermore, task decomposition can also help LLM agents handle tasks that '
 'requi

In [80]:
# Step 1: Generate a synthetic document using HyDE
generated_doc = generate_docs_for_retrieval.invoke({"question": question})

# Step 2: Retrieve relevant documents using the retriever
retrieved_docs = retriever.invoke(generated_doc)

# Output retrieved documents
retrieved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [81]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

pprint.pprint(final_rag_chain.invoke({"context":retrieved_docs,"question":question}))

('Task decomposition for LLM agents involves breaking down complex tasks into '
 'smaller and simpler steps to enhance model performance. This can be achieved '
 'through techniques like Chain of Thought (CoT) and Tree of Thoughts, which '
 'prompt the model to think step by step and explore multiple reasoning '
 'possibilities at each step. Task decomposition can also be done through '
 'simple prompting, task-specific instructions, or with human inputs.')
